# คำเเนะนำ

*   ควรจะโหลดข้อมูลให้ครบ
*   เชื่อม google drive




**สิ่งที่จะใช้**

*    Embedding model คือ BAAI/bge-base-en
*    vector database  ของ Chroma db
*    RAG ของ langchain


ทำการเชื่อม google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


โหลดสิ่งที่จำเป็น

In [ ]:
!pip install transformers torch chromadb
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q langchain transformers chromadb requests
!pip install -q langchain chromadb requests

In [ ]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q langchain transformers chromadb requests
!pip install -q langchain chromadb requests


ใช้โมเดล BGE-M3  ในการ Embed

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import json

# 🚀 โหลดโมเดล BGE-M3 (จาก BAAI)
model_name = 'BAAI/bge-base-en'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()  # ตั้งเป็นโหมดประเมินผล (ประหยัดหน่วยความจำ)

# 🔥 ฟังก์ชันที่ใช้ในการ embedding ข้อมูล
def embed_recommendations(categorized_recommendations):
    for idx, rec in enumerate(categorized_recommendations, start=1):
        text = rec['text']
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

        with torch.no_grad():
            embeddings = model(**inputs).last_hidden_state.mean(dim=1)

        rec['embedding'] = embeddings[0].cpu().numpy().tolist()
        print(f"✅ ข้อความที่ {idx} ได้รับการ embed แล้ว")
    return categorized_recommendations

# 💾 ฟังก์ชันที่ใช้ในการบันทึกข้อมูลลงในไฟล์ JSON
def save_embeddings_to_file(data, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"✅ บันทึกไฟล์ที่มีข้อมูล embedding ลงใน: {output_file_path}")

# 📥 ฟังก์ชันที่ใช้ในการอ่านข้อมูลจากไฟล์ JSON
def read_recommendations(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# 🚀 Main
file_path = '/content/drive/MyDrive/LockLearn/recommendations_with_categories.json'  # ระบุ path ของไฟล์ที่จัดหมวดหมู่แล้ว
recommendations = read_recommendations(file_path)

# 🔥 ทำการ embedding ข้อมูลคำแนะนำ
recommendations_with_embeddings = embed_recommendations(recommendations)

# 💾 บันทึกข้อมูลที่ embedding ลงในไฟล์แยก
embedding_output_file = '/content/drive/MyDrive/LockLearn/recommendations_with_embeddings_v3_BGE-M3.json'
save_embeddings_to_file(recommendations_with_embeddings, embedding_output_file)

print("✅ เสร็จสิ้นการทำ embedding ด้วย BGE-M3 และบันทึกข้อมูลลงไฟล์แยก")


สร้าง vector database

In [ ]:
import chromadb
import json

# 🚀 โหลดไฟล์ embeddings ที่มีอยู่แล้ว (BGE-M3)
embedding_file = '/content/drive/MyDrive/LockLearn/recommendations_with_embeddings_v3_BGE-M3.json'
with open(embedding_file, 'r', encoding='utf-8') as f:
    recommendations_with_embeddings = json.load(f)

# 🚀 สร้าง client และ collection (ใช้ Database เวอร์ชัน 3)
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database_v3'  # ระบุ path ของฐานข้อมูลใหม่
client = chromadb.PersistentClient(path=db_path)

# ลบ Collection เก่า (ถ้ามี)
if "recommendations" in [col.name for col in client.list_collections()]:
    client.delete_collection(name="recommendations")
    print("✅ ลบ Collection เก่าชื่อ 'recommendations' แล้ว")

# สร้าง Collection ใหม่
collection = client.get_or_create_collection(name="recommendations")
print("✅ สร้าง Collection ใหม่ชื่อ 'recommendations' ใน Chroma DB")

# 🚀 เพิ่มข้อมูลที่มี embedding ลงใน Chroma DB
print("🔄 กำลังเพิ่มคำแนะนำลงใน Chroma DB...")

for i, rec in enumerate(recommendations_with_embeddings):
    collection.add(
        documents=[rec['text']],
        embeddings=[rec['embedding']],
        metadatas=[{"category": rec.get('category', 'Unknown')}],
        ids=[f"rec_{i}"]
    )

print("✅ เก็บคำแนะนำทั้งหมดที่มี embedding ลง Chroma DB ใน Google Drive เรียบร้อย")
